In [139]:
!python --version

Python 3.8.10


In [140]:
!pip install transformers

You should consider upgrading via the '/home/tom/pyenv/beautifulSoup/bin/python -m pip install --upgrade pip' command.


In [141]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from transformers import EarlyStoppingCallback



# Define pretrained tokenizer and model
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)




loading file https://huggingface.co/bert-base-uncased/resolve/main/vocab.txt from cache at /home/tom/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/bert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer_config.json from cache at /home/tom/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading file https://huggingface.co/bert-base-uncased/resolve/main/tokenizer.json from cache at /home/tom/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6

In [142]:
FILE_PATH = './data/Twitter_Data.csv'
num_labels = 1

In [143]:
# config = BertConfig.from_pretrained(model_name)
# config.num_labels = 3

model = BertForSequenceClassification.from_pretrained(model_name, num_labels = num_labels)
# model = BertForSequenceClassification.from_pretrained(config)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /home/tom/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.11.3",
  "type_vocab_si

In [144]:
# Read data
# FILE_PATH = '/content/drive/MyDrive/Colab Notebooks/dataset/training.1600000.processed.noemoticon.csv'

DATASET_COLUMNS = ["text", "target"]
DATASET_ENCODING = "ISO-8859-1"


df = pd.read_csv(FILE_PATH
                 , encoding =DATASET_ENCODING
                 , names=DATASET_COLUMNS
                 , header=0)


In [145]:
df

,text,target
0,when modi promised âminimum government maxim...,-1.0
1,talk all the nonsense and continue all the dra...,0.0
2,what did just say vote for modi welcome bjp t...,1.0
3,asking his supporters prefix chowkidar their n...,1.0
4,answer who among these the most powerful world...,1.0
...,...,...
162971,why these 456 crores paid neerav modi not reco...,-1.0
162972,dear rss terrorist payal gawar what about modi...,-1.0
162973,did you cover her interaction forum where she ...,0.0
162974,there big project came into india modi dream p...,0.0


In [146]:
df.target = pd.to_numeric(df.target)

In [147]:
RANDOM_SEED = 42
MAX_LENGTH=100
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

# def decode_sentiment(input):
#     # return decode_map[int(label)]
#     return int(input/4)

# def decode2_sentiment(input):
#     return int(str(input))

df.info()

# newTarget = df.target.apply(lambda x: decode2_sentiment(x))



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162976 entries, 0 to 162975
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text    162976 non-null  object 
 1   target  162971 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [148]:
defaultValue = {'text':'', 'target': 0}
df = df.fillna(value=defaultValue)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162976 entries, 0 to 162975
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text    162976 non-null  object 
 1   target  162976 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [149]:
max(df.target)

1.0

In [150]:
trainRatio = 0.9
testRatio = 1 - trainRatio

In [151]:
# df_train_val, df_a = train_test_split(df, test_size=0.002, random_state=RANDOM_SEED)
# df_train_val, df_a = train_test_split(df, test_size=trainRatio, random_state=RANDOM_SEED)

In [152]:
X = list(df.text)
y = list(df.target)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=testRatio)
# X_train, y_train =  df.text, df.target


In [153]:
# df_bb, df_b = train_test_split(df, test_size=testRatio, random_state=RANDOM_SEED)
# XX = list(df_b.text)
# YY = list(df_b.target)
# XX_tokenize = tokenizer(XX, padding=True, truncation=True, max_length=MAX_LENGTH)
# YY[:20]

In [154]:
XX = list(df.text)
YY = list(df.target)
XX



['when modi promised â\x80\x9cminimum government maximum governanceâ\x80\x9d expected him begin the difficult job reforming the state why does take years get justice state should and not business and should exit psus and temples',
 'talk all the nonsense and continue all the drama will vote for modi ',
 'what did just say vote for modi  welcome bjp told you rahul the main campaigner for modi think modi should just relax',
 'asking his supporters prefix chowkidar their names modi did great service now there confusion what read what not now crustal clear what will crass filthy nonsensical see how most abuses are coming from chowkidars',
 'answer who among these the most powerful world leader today trump putin modi may ',
 'kiya tho refresh maarkefir comment karo ',
 'surat women perform yagna seeks divine grace for narendra modi become again\n',
 'this comes from cabinet which has scholars like modi smriti and hema time introspect',
 'with upcoming election india saga going important pai

In [155]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162976 entries, 0 to 162975
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   text    162976 non-null  object 
 1   target  162976 non-null  float64
dtypes: float64(1), object(1)
memory usage: 2.5+ MB


In [156]:
# for i,x in enumerate(XX):
#     try:
# #         print(f'{i}:{x}\n{tokenizer(x,padding=True, truncation=True, max_length=MAX_LENGTH)}')\
#         print('.', end='')
#     except Exception as e:
#         print(f'{i}:{x}')
#         print(f'Error {e.toString()}\n {x}')
        

In [157]:
# XX_tokenize = tokenizer(XX, padding=True, truncation=True, max_length=MAX_LENGTH)

In [158]:
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=MAX_LENGTH)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=MAX_LENGTH)

In [159]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [160]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [161]:
teset_dataset = Dataset(XX_tokenize)

In [165]:
# ----- 2. Fine-tune pretrained model -----#
# Define Trainer parameters
def compute_metrics(p):
    pred, labels = p
    print(pred,labels)
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred)
    precision = precision_score(y_true=labels, y_pred=pred)
    f1 = f1_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}


In [166]:
# Define Trainer
args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=100,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    seed=0,
    load_best_model_at_end=True,
)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [167]:
# Train pre-trained model
trainer.train()

***** Running training *****
  Num examples = 146678
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 36670


ValueError: Target size (torch.Size([8])) must be the same as input size (torch.Size([8, 3]))

In [ ]:
# ----- 3. Predict -----#
# Load test data
model_path = "output/checkpoint-800"
model = BertForSequenceClassification.from_pretrained(model_path, num_labels=num_labels)


loading configuration file output/checkpoint-800/config.json
Model config BertConfig {
  "_name_or_path": "bert-base-uncased",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.11.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file output/checkpoint-800/pytorch_model.bin
All model checkpoint weights were used when initializing BertForSequenceClassification.

All the weights of BertForSequenceCla

In [ ]:
test_trainer = Trainer(model)
raw_pred, _, _ = test_trainer.predict(teset_dataset)
y_pred = np.argmax(raw_pred, axis=1)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Prediction *****
  Num examples = 3200
  Batch size = 8


In [ ]:
y_pred[:20]

array([1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0])

In [ ]:
YY[:20]

[0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0]

In [ ]:
a_s= accuracy_score(y_true=YY, y_pred=y_pred)
a_s

0.8778125

In [ ]:
PATH = f"/content/drive/MyDrive/Colab Notebooks/models/210930_a{str(a_s)[2:]}/"
torch.save(model.state_dict(), PATH + "state_dict.pt")


In [ ]:
! ls -all "$PATH"

total 427754
-rw------- 1 root root 438019245 Sep 29 18:35 state_dict.pt
